#Installing open brewery lib

In [0]:
pip install openbrewerydb

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached openbrewerydb-0.1.1-py2.py3-none-any.whl (4.5 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


#Importing required libs

In [0]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col, current_timestamp, xxhash64, concat_ws
from delta.tables import * 

#Getting API data and declaring df schema

In [0]:
url = "https://api.openbrewerydb.org/breweries"

response = requests.get(url)
data = response.json()

schema = StructType(fields=[ 
    StructField('id', StringType(), True),
    StructField('name', StringType(), True),
    StructField('brewery_type', StringType(), True),
    StructField('address_1', StringType(), True),
    StructField('address_2', StringType(), True),
    StructField('address_3', StringType(), True),
    StructField('city', StringType(), True),
    StructField('state_province', StringType(), True),
    StructField('postal_code', StringType(), True),
    StructField('country', StringType(), True),
    StructField('longitude', StringType(), True),
    StructField('latitude', StringType(), True),
    StructField('phone', StringType(), True),
    StructField('website_url', StringType(), True),
    StructField('state', StringType(), True),
    StructField('street', StringType(), True)
    ])

df = spark.createDataFrame(data, schema=schema)


#Transformation
Adding a timestamp for data insertion control.

In [0]:
  df = df.withColumn("insertion_at", current_timestamp())

In [0]:
tableFullName = 'bronze.open_brewery_db.breweries_list'

#Creating if not exists delta table

In [0]:
DeltaTable.createIfNotExists(spark) \
  .tableName(tableFullName) \
  .addColumns(df.schema) \
  .execute()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-140001555563394>, line 4
      1 DeltaTable.createIfNotExists(spark) \
      2   .tableName(tableFullName) \
      3   .addColumns(df.schema) \
----> 4   .execute()

File /databricks/spark/python/delta/tables.py:1534, in DeltaTableBuilder.execute(self)
   1525 @since(1.0)  # type: ignore[arg-type]
   1526 def execute(self) -> DeltaTable:
   1527     """
   1528     Execute Table Creation.
   1529 
   (...)
   1532     .. note:: Evolving
   1533     """
-> 1534     jdt = self._jbuilder.execute()
   1535     return DeltaTable(self._spark, jdt)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *args)
   1349 command = proto.CALL_COMMAND_NAME +\
   1350     self.command_header +\
   1351     args_command +\
   1352     proto.END_COMMAND_PART
   1354 answer = self.g

#Saving Delta Table

In [0]:
df.write.mode("append").saveAsTable(f"{tableFullName}")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-140001555563397>, line 1
----> 1 df.write.mode("append").format("parquet").saveAsTable(f"{tableFullName}")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1841, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1839 if format is not None:
   1840     self.format(format)
-> 1841 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(s